# 9. Incorporating OD Veto Data

In [1]:
import sys
import os
import h5py
from collections import Counter
from progressbar import *
import re
import numpy as np
import h5py
from scipy import signal
import matplotlib
from repeating_classifier_training_utils import *
from functools import reduce

# Add the path to the parent directory to augment search for module
par_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

if par_dir not in sys.path:
    sys.path.append(par_dir)
%load_ext autoreload
%matplotlib inline
%autoreload 2

In [2]:
veto_path = '/fast_scratch/WatChMaL/data/IWCDmPMT_4pm_full_tank_ODveto.h5'

In [3]:
odv_file = h5py.File(veto_path,'r')
odv_info = {}
for key in odv_file.keys():
    odv_info[key] = np.array(odv_file[key])

In [15]:
odv_dict = {}
pbar = ProgressBar(widgets=['Creating Event-Index Dictionary: ', Percentage(), ' ', Bar(marker='0',left='[',right=']'),
           ' ', ETA()], maxval=len(odv_info['event_ids']))
pbar.start()
for i in range(len(odv_info['event_ids'])):
    odv_dict[(odv_info['root_files'][i], odv_info['event_ids'][i])] = i
    pbar.update(i)
pbar.finish()

Creating Event-Index Dictionary: 100% [0000000000000000000000000] Time: 0:00:15


## Load test set

In [12]:
# Get original h5 file info
# Import test events from h5 file
filtered_index = "/fast_scratch/WatChMaL/data/IWCD_fulltank_300_pe_idxs.npz"
filtered_indices = np.load(filtered_index, allow_pickle=True)
test_filtered_indices = filtered_indices['test_idxs']

original_data_path = "/data/WatChMaL/data/IWCDmPMT_4pi_fulltank_9M.h5"
f = h5py.File(original_data_path, "r")

original_eventids = np.array(f['event_ids'])
original_rootfiles = np.array(f['root_files'])

filtered_eventids = original_eventids[test_filtered_indices]
filtered_rootfiles = original_rootfiles[test_filtered_indices]

In [18]:
odv_mapping_indices = np.zeros(len(filtered_rootfiles))
pbar = ProgressBar(widgets=['Mapping Progress: ', Percentage(), ' ', Bar(marker='0',left='[',right=']'),
           ' ', ETA()], maxval=len(filtered_rootfiles))
pbar.start()
for i in range(len(filtered_rootfiles)):
    odv_mapping_indices[i] = odv_dict[(filtered_rootfiles[i], filtered_eventids[i])]
    pbar.update(i)
pbar.finish()
odv_mapping_indices = np.int32(odv_mapping_indices)

Mapping Progress: 100% [0000000000000000000000000000000000000000] Time: 0:00:09


In [23]:
pbar = ProgressBar(widgets=['Verification Progress: ', Percentage(), ' ', Bar(marker='0',left='[',right=']'),
           ' ', ETA()], maxval=len(filtered_rootfiles))
pbar.start()
for i in range(len(filtered_rootfiles)):
    assert odv_info['root_files'][odv_mapping_indices[i]] == filtered_rootfiles[i]
    assert odv_info['event_ids'][odv_mapping_indices[i]] == filtered_eventids[i]
    pbar.update(i)
pbar.finish()

Verification Progress: 100% [00000000000000000000000000000000000] Time: 0:00:13


In [27]:
np.savez(os.path.join(os.getcwd(), 'Index_Storage/od_veto_mapping_idxs.npz'), mapping_idxs_full_set=odv_mapping_indices)